**ReadMe**

The part 1 is a discussion on the use of tabular algorithm.

The question 1 is answered in part 2.

The question 2 is answered in part 2.

The question 3 is answered in part 2.

In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
%cd /content/drive/My\ Drive/Winter\ 2022/CME241

Mounted at /content/drive
/content/drive/My Drive/Winter 2022/CME241


In [37]:
from rl.distribution import *
from rl.function_approx import *
from rl.markov_decision_process import *
from rl.policy import *
from rl.dynamic_programming import *
from rl.approximate_dynamic_programming import * 
from rl.monte_carlo import *
from rl.chapter2.simple_inventory_mrp import *
import numpy as np
from itertools import islice
from rl.gen_utils.plot_funcs import plot_list_of_curves
import matplotlib.pyplot as plt
import time
from typing import Tuple, Sequence, Iterator, List
from rl.td import *

#1. Why using Tabular \& Limitations? 

In the second part of this assignment, we will deal with the case of finite Markov reward process. Although the fact that the process is finite does not necessarily force us to use tabular algorithm in approximating the dynamics, it is one of the most straightforward ways to understand the expected reward while also not implicitly enforcing any parametric guess on the form of the predicted value.[link text](https://)

The tabularness of the prediction will allow us to treat each data input (which has finite variations) as an almost disconnected value in the sense that, even if this data input is similar to another in the input space, we will not enforce any guess that it will also look similar. Although this assumption of non-assuming similarity for similar input may sounds bad for our prediction since we expect similar things to yield similar behavior, this may not be true especially when we deal with a system with high sensitivity in which small change in the input can result in a tremendous change in the output. Moreover, assuming dependence upon neighbor can also introduce bias into the model, as what we predict for this case will rather also use the observation from other cases.

Another use case for the tabular algorithm is when we do not know about the input. For example, we may not know how to measure the distance or the similarity of the inputs.

For example, if we want to estimate the willing to pay prices for our products, which are cars, chopsticks, and sneakers. It is possible likely that sneakers maybe more similar to cars than to chopsticks but it is also quite possible in the differ\ence case. The similarity in this sense may be hard to qunatify and hard to be used. Therefore, even though we have several datapoints for bith cars and chopsticks but a very few datapoints for sneakers, we may not want to interpolate the relationship from the cars and chopsticks into the sneakers.

However, this market model has been heavily challenged by the recent growth of online markets.

We can see that in online marketplace, although the total number of items are still finite, the number can be very large. Moreover, the purchase can be sparse. (When a person tries to buy some items from Amazon, he may search on the product he wants to look into and gets to see hundreds of products. He eventually choose to purchase only one of them or maybe non of them.)  Therefore, if we treat each item as a discrete point without any interaction, we may get a very unreliable model. Moreover, as the end objective of the consumer demand estimation is not to actually know the demand, but to rather use the demand to understand what will happen if we launch a new product of this characteristics or what will ahppen if we put some promotion upon this products. We want to get a counterfactual prediction, in which the product we ask the model to predict for us has never been seen by the model before. 

(We say that imposing a promotion can be treated as a new product as the price will change so the product can be said to be difference. In some model consideration, people will only discretize the item input by only the phsyical characteristics but not the price.)

One technique that we can do is to just think about what is the most similar product to this one and use the same value as a prediction. Another naive way is to find k most similar product and take an average or a weughted average. This is indeed the same as KNN (k-Nearest Neighbor). In KNN such as in image classifier, the input is an image that we can quantify as a vector of pixel values. However, in the training process, we do not use those value at all, and we do proceed to evaluate each item individually. Since we discretize directly by putting image with exacty the same vector representation into 1 point (in the table), we will see that one table will only see 1 instance! (since it is quite hgard for two images to have the same vector representations in the highdimensional vector space). Thus, the training is to observe that 1 value per 1 input in the table, the best way to do so is then to take taht value as the predicted value. Therefore, the trainign of the KNN does not require any actual computation, but only requires the storing of the big table of the same size as teh data input itself.

In the evaluation stage, we input an image to the KNN. It is likelythat the image we ask the model to predict will not have any overlapping with the training data point, so we fidn the k-nearest neghbor and find the average prediction instead.

For example, let consider an easier to understand case.

If we try to predict a curvature of a two dimensional curve at different x value.

If we want to assume some regulation on the conitnuity of the curve, we will see that if we perturb the value of x to be $x+\delta x$, we will expect to see that the crosssections at the two x values will be only alittle bit different. Therefore, we can use the data of observation at x to help the prediction at $x+\delta x$ even though it is not at the exactlyt same spot. For example, one can do so by adding regularization while training.

Note that in the neural networks model where we impose the L2 regularization can also be thought of that we want similar input to not have so stark difference. For example if the model is
\begin{align*}
  y = wx
\end{align*}
Therefore,
\begin{align*}
  \vert \Delta y \vert  = \vert w \vert \vert \Delta x \vert 
\end{align*}

Thus, as the regularization tries to push the value of $w$ to be closer top zero. The relative difference will also be smaller.

This, therefore, will help reducing variance in the prediction. However, if we want to model a system where we have already know that there should be a great variation (such as modelling rectangular wave), the regularization can lead to serious bias.

For physics example, we can see that the nature has a regularity enforcement upon most mechanical system. For example, the speed cannot be faster than the speed of light. Therefore, we will know for sure that the displacement of an object of interests cannot be more than $ct$.

Therefore, if we observe the location of a car at time = 8.00am, we will be able to bound the possible location of the car at time = 1.00pm. 

The question is that will we use that bound to guide our prediction. 

The answer is clearly "No."

Within that time span of three hours, the car (if only limited by the speed of lights, can go back and forth from the earth to the sun for more than ten times! Therefore, this bound will says that any place in the world is possible.

#2. Simple Inventory Self-Dynamics via Reinforcement Learning

We, first, denote that this problem will be a case when we do not impose any intervention into the system. In other words, we will not submit any more order for the store. Therefore, we will be able to observe the self-dynamics of the inventory.

We first initialize the simple inventory model.

In [3]:
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9

si_mrp = SimpleInventoryMRPFinite(
        capacity=user_capacity,
        poisson_lambda=user_poisson_lambda,
        holding_cost=user_holding_cost,
        stockout_cost=user_stockout_cost
    )

We first take a look at the dybnamics, and can see the cyclical structure. In other words, we can observe any state by starting at anypoint given that we have enough time.

In [4]:
si_mrp

From State InventoryState(on_hand=0, on_order=0):
  To [State InventoryState(on_hand=0, on_order=2) and Reward -10.000] with Probability 1.000
From State InventoryState(on_hand=0, on_order=1):
  To [State InventoryState(on_hand=1, on_order=1) and Reward -0.000] with Probability 0.368
  To [State InventoryState(on_hand=0, on_order=1) and Reward -3.679] with Probability 0.632
From State InventoryState(on_hand=0, on_order=2):
  To [State InventoryState(on_hand=2, on_order=0) and Reward -0.000] with Probability 0.368
  To [State InventoryState(on_hand=1, on_order=0) and Reward -0.000] with Probability 0.368
  To [State InventoryState(on_hand=0, on_order=0) and Reward -1.036] with Probability 0.264
From State InventoryState(on_hand=1, on_order=0):
  To [State InventoryState(on_hand=1, on_order=1) and Reward -1.000] with Probability 0.368
  To [State InventoryState(on_hand=0, on_order=1) and Reward -4.679] with Probability 0.632
From State InventoryState(on_hand=1, on_order=1):
  To [State I

However, in some other cases, there can be several source, which we cannot observe it again after we have observe it once, or there may be different part of the system where starting from a source cannot lead to all possible states. We therefore assume that each non terminal state is equally chosen from the pool of non-terminal state of the markov reward process. In some sense, this method may be inefficient. However, it will always ensure that each state will be observed. Therefore, this is quite crucial for tabular method since we enforce no regularization between each state's value directly.

## 2.1 Monte Carlo Method

First, we try to define the count_to_weight_func.

In [5]:
def count_to_weight_func(n):
  return 1/n

We can also try to do so by using lambda

In [6]:
count_to_weight_func = lambda n: 1/ n

#Test
count_to_weight_func(6)

0.16666666666666666

Since we want to run MC, we have to first sample an episode and find the return of each popint in the episode. 

As our system does not have any sink, the chain does go on forever. We therefore have to use the tool of value_tolerance (this can be done since user_gamma = 0.9 < 1).

The function we use wkill be returns.

This function will take in the stream of reward trace and find the value such that the gamma to the power of it becomes less than the tolerance. Afterwards, the function will keep only two time of that value as the size of the sequence of step and rewards. The function will then perform the accumulation of rewards and return the return of each state (in the first half of the terminated sequence)

In [7]:
reward_traces_gen = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

#get episode
episode = returns(next(reward_traces_gen), user_gamma, tolerance=1e-6)

In [8]:
#Initialize value of each non-terminal state as a dict
V_tabular: Dict[[NonTerminal], float] = dict()
count_tabular: Dict[[NonTerminal], int] = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  V_tabular[non_terminal_state] = 0
  count_tabular[non_terminal_state] = 0

for step in episode:
  #identify the current non-terminal state
  non_terminal_state = step.state
  #update the count
  count_tabular[non_terminal_state] += 1
  #update the value
  temp_f = count_to_weight_func(count_tabular[non_terminal_state])
  V_tabular[non_terminal_state] = ((1-temp_f)*V_tabular[non_terminal_state]) + (temp_f*(step.return_))

Now, we can concatenate the framework together

In [9]:
#Define count_to_weight_func
count_to_weight_func = lambda n: 1/ n

#Sepecify number of episode we are given
num_episodes = 1000

#Create reward traces
reward_traces_gen = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

#Initialize value of each non-terminal state as a dict
V_tabular: Dict[[NonTerminal], float] = dict()
count_tabular: Dict[[NonTerminal], int] = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  V_tabular[non_terminal_state] = 0
  count_tabular[non_terminal_state] = 0

for _ in range(num_episodes):
  #Get episode
  episode = returns(next(reward_traces_gen), user_gamma, tolerance=1e-6)

  #Use every step in the terminated episode
  for step in episode:
    #identify the current non-terminal state
    non_terminal_state = step.state
    #update the count
    count_tabular[non_terminal_state] += 1
    #update the value
    temp_f = count_to_weight_func(count_tabular[non_terminal_state])
    V_tabular[non_terminal_state] = ((1-temp_f)*V_tabular[non_terminal_state]) + (temp_f*(step.return_))

In [10]:
V_tabular

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.51194237017225,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.94854059105788,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.351705815341017,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.947159807473067,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.36572727768168,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.40777982255558}

We can then try to compare the value with the framework we have given.

In [11]:
#Sepecify number of episode we are given
num_episodes = 1000

#Create reward traces
reward_traces_gen = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

mc_V_tabulars = mc_prediction(reward_traces_gen, Tabular(count_to_weight_func = lambda n: 1/ n), user_gamma)

#Note that the first next does not yiled anything useful (due to the design of the code)
for _ in range(num_episodes):
  next(mc_V_tabulars)

mc_V_tabular = next(mc_V_tabulars)

In [12]:
mc_V_tabular.values_map

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.570323673178756,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.938426994942077,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.418909861059895,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.940955438526046,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.371675647976833,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.41068878359819}

## 2.2 Temporal Difference Method

We have seen that in Monte Carlo method, we have to wait until the episode is ended and uodate the value function. This implication makes Monte Carlo method undesirable in some use cases despite of its straightforwardness.

We will first try to work with 1 episode.

In [13]:
#Specify alpha (learning rate)
alpha = 0.1

#Create reward traces
reward_traces_gen = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

#Get reward trace of 1 episode
#Note that we dont have to calculate the return of each point in the trace
reward_trace = next(reward_traces_gen)

Similarly, we will create a V function.

However, we do not have to keep track of counts.

In [14]:
#Initialize value of each non-terminal state as a dict
td_V_tabular: Dict[[NonTerminal], float] = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  td_V_tabular[non_terminal_state] = 0

Next, we will try to find what we want from one episode. Since one episode can run on forever, we have to set some stop for it.

In [15]:
max_steps = round(np.log(1e-6) / np.log(user_gamma)) 

In [23]:
i = 0
for step in reward_trace:
  if i >= max_steps:
    break
  i += 1
  td_V_tabular[step.state] = td_V_tabular[step.state] + alpha * (step.reward + user_gamma * td_V_tabular[step.next_state] - td_V_tabular[step.state])

In this case, since we know for sure that it will run forever without interruption, we can chane to be iteration over i.

In [22]:
for _ in range(max_steps):
  step = next(reward_trace)
  td_V_tabular[step.state] = td_V_tabular[step.state] + alpha * (step.reward + user_gamma * td_V_tabular[step.next_state] - td_V_tabular[step.state])

Afterwards, we can concatenate everything together.

In [33]:
#Specify alpha (learning rate)
alpha = 0.5

#Sepecify number of episode we are given
num_episodes = 1000

#Specify max steps per episode to be used
max_steps = round(np.log(1e-6) / np.log(user_gamma)) 

#Create reward traces
reward_traces_gen = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

#Initialize value of each non-terminal state as a dict
td_V_tabular: Dict[[NonTerminal], float] = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  td_V_tabular[non_terminal_state] = 0

for _ in range(num_episodes): 
  #Get episode
  reward_trace = next(reward_traces_gen)
  for _ in range(5):
    step = next(reward_trace)
    td_V_tabular[step.state] = td_V_tabular[step.state] + alpha * (step.reward + user_gamma * td_V_tabular[step.next_state] - td_V_tabular[step.state])

In [35]:
td_V_tabular

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -37.13597759567777,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -29.482079653217838,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.46355159613232,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -29.79350612177187,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -31.01063420393296,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -34.081303020840934}

Next, we try to compare it with td_prediction framework

In [92]:
#Sepecify number of episode we are given
num_episodes = 1000

#Create reward traces
reward_traces_gen = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))
td_experiences = itertools.chain.from_iterable(itertools.islice(episode, max_steps) for episode in reward_traces_gen)

#Specify max steps per episode to be used
max_steps = round(np.log(1e-6) / np.log(user_gamma)) 

td_V_tabulars_ = td_prediction(td_experiences, Tabular(count_to_weight_func=lambda n: 0.5), user_gamma)

for _ in range(num_episodes*max_steps):
  next(td_V_tabulars_)

td_V_tabular_ = next(td_V_tabulars_)

In [91]:
td_V_tabular_.values_map

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -36.00128786420203,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -38.79654310726185,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -17.373573643260446,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -40.019720772830475,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -25.770576265106047,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.248607157624576}